# reLAISS Tutorial 01
### written by Evan Reynolds and Alex Gagliano

reLAISS is the second version of LAISS (Lightcurve Anomaly Identification & Similarity Search); a tool to find similar supernovae & identify anomalous supernovae (and the galaxies that host them) using their photometric features. reLAISS introduces several new functionalites, some of which will be presented in this tutorial.

The similarity search takes advantage of [Approximate Nearest Neighbors Oh Yeah (ANNOY)](https://github.com/spotify/annoy), the approximate nearest neighbors algorithm developed by Spotify that allows you to come up with a relevant song to listen to before your current one ends. The anomaly detection classifier is an isolation forest model trained on a dataset bank of (currently) 22,000 transients.

Let's start by installing and loading the relevant packages:

In [ ]:
# TODO: Code to install necessary packages. Maybe Alex can help?

In [ ]:
import os
import sys

import warnings
warnings.filterwarnings("ignore")

working_dir = os.path.abspath("")
os.chdir(working_dir)
sys.path.append('../code/')

import constants
from helper_func import *
from relaiss_func import *

Let's also download and unpack the data we need for this tutorial: 

In [ ]:
# TODO: Download sfd data, dataset bank (and theorized_lc data, if you want to show this)
# Alex to help with this? Make sure data is in the right spot for the rest of the tutorial

Now, we must choose the features of both the host galaxy and the supernova lightcurve to use in our matching algorithm & anomaly classifer. Open the constants.py file in the "code" directory and review the lightcurve and host features. Comment out any features you'd like to exclude. Then, restart the notebook kernel, reimport packages, and run the code below.

In [ ]:
lc_feature_names = constants.lc_features_const.copy()
host_feature_names = constants.host_features_const.copy()

The rest is taken care of by our main function, `re_LAISS`. There are many inputs to the function, but here are the basic ones:
* `path_to_dataset_bank`: The path from your working directory to the dataset bank downloaded earlier.
* `transient_ztf_id`: The ZTF ID of the supernova used in the similarity search and anomaly detection. 
* `lc_feature_names`: The names of the lightcurve features to be used, which we defined earlier.
* `host_feature_names`: The names of the host galaxy features to be used, which we defined earlier.
* `path_to_sfd_data_folder`: The path from your working directory to the sfd data folder downloaded earlier.
* `neighbors`: The number of "nearest neighbors" to return from the similarity search.
* `num_mc_simulations`: reLAISS can resample the input feature set using Monte Carlo simulation with feature errors. This is the number of simulations it will use. Set to 0 to turn off this feature.
* `weight_lc_feats_factor`: A factor that multiplicatively weights the effect of the lightcurve features.
* `suggest_neighbor_num`: A boolean that, if true, will show a plot of neighbors and distances from the input supernova and suggest a number of neighbors to look at.
* `run_AD`: A boolean indicating whether or not to run the anomaly detection model.
* `run_NN`: A boolean indicating whether or not to run the similarity search.
*  `save_figures`: A boolean indicating whether or not to save all figures created while reLAISS runs. If true, directories will be created and figured will automatically be organized.

As for what it returns, note that reLAISS must have at least one of `run_NN`, `run_AD`, or `suggest_neighbor_num` equal to true for it to run. Otherwise, it does nothing. If `run_NN` or `suggest_neighbor_num` is true, reLAISS will return objects `neighbors_df`, `primer_dict`. If only `run_AD` is true, reLAISS will return nothing.
* `neighbors_df` is a pandas dataframe of the nearest neighbors identified for the input supernova. This is only populated if `run_NN` is true. 
* `primer_dict` is a dictionary containing relevant information about the input supernova.

Here's a basic workflow you might go through. Let's say you're interested in the supernova "ZTF21aaublej". First, let's run re_LAISS with only `suggest_neighbor_num=True` and a large number of neighbors, to get an idea of the distance curve. We'll leave other arguments to their default values.

In [ ]:
neighbors_df, primer_dict = re_LAISS(
    transient_ztf_id='ZTF21aaublej',
    path_to_dataset_bank='../data/df_bank.csv',
    path_to_sfd_data_folder='../data/sfddata-master',
    host_feature_names=host_feature_names,
    lc_feature_names=lc_feature_names,
    neighbors=40,
    suggest_neighbor_num=True,
    run_NN=False,
    run_AD=False,
    save_figures=True,
)

You'll see that some files were created for the similarity search. And, our neighbor distance plot is displayed. It looks like anywhere from 5 to 7 neighbors is a reasonable cutoff, so let's use 7 and run the full neighbor search by setting `suggest_neighbor_num=False` and `run_NN=True`.

In [ ]:
neighbors_df, primer_dict = re_LAISS(
    transient_ztf_id='ZTF21aaublej',
    path_to_dataset_bank='../data/df_bank.csv',
    path_to_sfd_data_folder='../data/sfddata-master',
    host_feature_names=host_feature_names,
    lc_feature_names=lc_feature_names,
    neighbors=7,
    suggest_neighbor_num=False,
    run_NN=True,
    run_AD=False,
    save_figures=False,
)

Congratulations on your first similarity search! You can see a plot of the input suprnova lightcurve, as well as the neighbor lightcurves. They're not a great match right now. And, you see a plot of the input supernova host galaxy, as well as the neighbor hosts. These look decently similar. So, let's say you wanted a better lightcurve matches for your supernova. You could try using Monte Carlo simulation or lightcurve upweighting. Or, you could pass in an empty host features list; this would run a "lighcurve-only" reLAISS that picks neighbors based only on lightcurve properties.

The code below uses 20 Monte Carlo simulations with upweighted lightcurve features. Run it and you should see that the matches look much better! Experimenting with parameters and feature sets is a key functionality of reLAISS that can help you find better matches.

In [ ]:
neighbors_df, primer_dict = re_LAISS(
    transient_ztf_id='ZTF21aaublej',
    path_to_dataset_bank='../data/df_bank.csv',
    path_to_sfd_data_folder='../data/sfddata-master',
    host_feature_names=host_feature_names,
    lc_feature_names=lc_feature_names,
    neighbors=7,
    suggest_neighbor_num=False,
    run_NN=True,
    run_AD=False,
    save_figures=False,
    num_mc_simulations=20,
    weight_lc_feats_factor=3,
)

Lastly, we'll run anomaly detection on the supernova of interest. This will produce a pdf of the lightcurve of the input supernova, as well as a graph of the probability (in time) that the supernova is anomalous. If the probability is ever over 50%, the supernova is flagged as anomalous.

As a reminder, if you're only running anomaly detection, reLAISS returns nothing.

In [ ]:
re_LAISS(
    transient_ztf_id='ZTF21aaublej',
    path_to_dataset_bank='../data/df_bank.csv',
    path_to_sfd_data_folder='../data/sfddata-master',
    host_feature_names=host_feature_names,
    lc_feature_names=lc_feature_names,
    suggest_neighbor_num=False,
    run_NN=False,
    run_AD=True,
    save_figures=False,
)

That concludes the tutorial on the basics of reLAISS! Below is an example of how to run the similarity search and anomaly detection at once, while saving important figures. Try it out if you'd like.

And, see notebook tutorial 02 for a walkthrough of some of the more advanced functionalities of reLAISS!

In [ ]:
neighbors_df, primer_dict = re_LAISS(
    transient_ztf_id='ZTF21aaublej',
    path_to_dataset_bank='../data/df_bank.csv',
    path_to_sfd_data_folder='../data/sfddata-master',
    host_feature_names=host_feature_names,
    lc_feature_names=lc_feature_names,
    neighbors=7,
    suggest_neighbor_num=False,
    run_NN=True,
    run_AD=True,
    save_figures=True,
    num_mc_simulations=20,
    weight_lc_feats_factor=3,
)